## Tratamento e Engenharia de Features
- [ ] xxx;

## Criação de Novos Datasets
- [ ] Dataset de alunos de SISTEMAS DE INFORMAÇÃO (registro por aluno) com uma coluna para média geral de todos os semestres e quantidade de faltas para todos os mesmos (realizar parse para cadeiras que nao existem mais) (information_systems.csv);
- [ ] Dataset de nota média de alunos para a cadeira mais frequente por período letivo;


In [1]:
import pandas as pd

df = pd.read_csv("datasets/original.csv")
df = df.sort_values(by=["student_id", "period", "discipline"])

df.head(10)

,id,student_id,birthdate,sex,city,course,period,week_day,discipline,status,g1,g2,final_grade,class_skips
2138,1,99951,1992-05-21 02:00:00.000 -0300,F,Faxinal do Soturno,Ciências Contábeis,2021/1,Quarta,Matemática Aplicada - 60,Aprovado,7.5,7.0,7.2,0
5665,3,99951,1992-05-21 02:00:00.000 -0300,F,Faxinal do Soturno,Ciências Contábeis,2021/2,Segunda,Contabilidade Intermediária - 60,Aprovado,7.2,8.9,8.0,0
9160,2,99951,1992-05-21 02:00:00.000 -0300,F,Faxinal do Soturno,Ciências Contábeis,2022/1,Terça,Contabilidade Avançada - 60,Trancado,-1.0,-1.0,-1.0,0
28710,4,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/1,Segunda,Contabilidade Introdutória - 60,Aprovado,8.4,8.8,8.6,0
28312,8,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/1,Quarta,Legislação e Ética Profissional - 30,Aprovado,9.8,9.9,9.8,0
28745,6,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/1,Quarta,Teoria da Contabilidade - 30,Aprovado,9.9,9.0,9.4,4
28311,9,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/1,Quinta,Teorias das Organizações - 60,Aprovado,10.0,8.6,9.3,8
33950,10,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/2,Sábado,Direito Empresarial - 60,Matriculado,-1.0,-1.0,-1.0,2
34055,7,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/2,Segunda,Estrutura das Demonstrações Contábeis - 60,Matriculado,-1.0,-1.0,-1.0,0
33988,5,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/2,Quarta,Finanças Corporativas I - 60,Matriculado,-1.0,-1.0,-1.0,0


In [2]:
original_df_len = len(df)
print(f"Quantidade de registros: {original_df_len}")

Quantidade de registros: 35995


In [3]:


# Tratamento da coluna "status"
df['status_normalized'] = df['status'].astype(str).str.lower().eq('aprovado').astype('int8')

def normalize_grade_column(colum_name: str):
    new_column_name = f"{ colum_name }_normalized"
    df[new_column_name] = pd.to_numeric(df[colum_name], errors='coerce') # valores inválidos viram NaN
    df[new_column_name] = (df[new_column_name] / 10).round(2) # Normalização e ajuste de decimais
    df.loc[df[new_column_name] == -0.1, new_column_name] = 0 # Valores "-0.1" trocados para 0

normalize_grade_column("g1")
normalize_grade_column("g2")
normalize_grade_column("final_grade")

# # Remoção do "traço" e do crédito da coluna disciplina no nome da mesma
df["discipline_normalized"] = df["discipline"].str.split(" - ").str[0].str.strip()
# Remoção para caso especial em "Gestão da Qualidade de Software"
df["discipline_normalized"] = df["discipline_normalized"].str.split(" -72").str[0].str.strip()

# Anonimização de disciplinas menos cursadas
# discipline
discipline_counts = df.groupby("discipline")["student_id"].nunique()
rare_disciplines = discipline_counts[discipline_counts <= 10].index
df["discipline"] = df["discipline"].where(~df["discipline"].isin(rare_disciplines), "Outro")
df = df[df["discipline"] != "Outros"]
# discipline_normalized
discipline_counts = df.groupby("discipline_normalized")["student_id"].nunique()
rare_disciplines = discipline_counts[discipline_counts < 10].index
df["discipline_normalized"] = df["discipline_normalized"].where(~df["discipline_normalized"].isin(rare_disciplines), "Outro")
df = df[df["discipline_normalized"] != "Outros"]

# Criação de "canceled_discipline" para registros com "status" trancado ou cancelado
df["canceled_discipline"] = df["status"].isin(["Trancado", "Cancelado"]).astype(int)

# Criação de "skipped_discipline" para registros com "status" reprovado por frequência
df["skipped_discipline"] = df["status"].isin(["Reprovado por Frequência"]).astype(int)

# Criação de "sex_normalized" para normalização do sexo biológico
df["sex_normalized"] = df["sex"].map({"M": 1, "F": 2}).fillna(0).astype(int)

for col in ["week_day", "course", "period"]:
    df[f"{ col }_normalized"] = pd.factorize(df[col])[0] + 1

# Criação de "birthdate_normalized" para a captura da data apenas (e não timestamp)
df["birthdate_normalized"] = df["birthdate"].str.split(" ").str[0].str.strip()

# Criação de "class_skips_normalized" para a normalização e ajuste das faltas por dia
df["class_skips_normalized"] = (df["class_skips"] / 4).astype(int)

# Salva dataset original com tratamentos
df.to_csv("datasets/original_and_treated.csv", index=False, encoding="utf-8")

df.head(10)

,id,student_id,birthdate,sex,city,course,period,week_day,discipline,status,...,final_grade_normalized,discipline_normalized,canceled_discipline,skipped_discipline,sex_normalized,week_day_normalized,course_normalized,period_normalized,birthdate_normalized,class_skips_normalized
2138,1,99951,1992-05-21 02:00:00.000 -0300,F,Faxinal do Soturno,Ciências Contábeis,2021/1,Quarta,Matemática Aplicada - 60,Aprovado,...,0.72,Matemática Aplicada,0,0,2,1,1,1,1992-05-21,0
5665,3,99951,1992-05-21 02:00:00.000 -0300,F,Faxinal do Soturno,Ciências Contábeis,2021/2,Segunda,Contabilidade Intermediária - 60,Aprovado,...,0.80,Contabilidade Intermediária,0,0,2,2,1,2,1992-05-21,0
9160,2,99951,1992-05-21 02:00:00.000 -0300,F,Faxinal do Soturno,Ciências Contábeis,2022/1,Terça,Contabilidade Avançada - 60,Trancado,...,0.00,Contabilidade Avançada,1,0,2,3,1,3,1992-05-21,0
28710,4,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/1,Segunda,Contabilidade Introdutória - 60,Aprovado,...,0.86,Contabilidade Introdutória,0,0,1,2,1,4,1989-01-06,0
28312,8,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/1,Quarta,Legislação e Ética Profissional - 30,Aprovado,...,0.98,Legislação e Ética Profissional,0,0,1,1,1,4,1989-01-06,0
28745,6,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/1,Quarta,Teoria da Contabilidade - 30,Aprovado,...,0.94,Teoria da Contabilidade,0,0,1,1,1,4,1989-01-06,1
28311,9,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/1,Quinta,Teorias das Organizações - 60,Aprovado,...,0.93,Teorias das Organizações,0,0,1,4,1,4,1989-01-06,2
33950,10,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/2,Sábado,Direito Empresarial - 60,Matriculado,...,0.00,Direito Empresarial,0,0,1,5,1,5,1989-01-06,0
34055,7,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/2,Segunda,Estrutura das Demonstrações Contábeis - 60,Matriculado,...,0.00,Estrutura das Demonstrações Contábeis,0,0,1,2,1,5,1989-01-06,0
33988,5,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/2,Quarta,Finanças Corporativas I - 60,Matriculado,...,0.00,Finanças Corporativas I,0,0,1,1,1,5,1989-01-06,0


In [4]:
original_and_treated_df_len = len(df)
print(f"Quantidade de registros: {original_and_treated_df_len}")
print(f"Registros removidos durante o tratamento: {original_df_len - original_and_treated_df_len}")

Quantidade de registros: 35995
Registros removidos durante o tratamento: 0


In [5]:
treated_df = df.copy(deep=True)

# Mantém só as colunas desejadas
cols = [
    "id", "student_id", "birthdate_normalized", "sex_normalized", "city",
    "course_normalized", "period_normalized", "week_day_normalized",
    "discipline_normalized", "status_normalized", "g1_normalized",
    "g2_normalized", "final_grade_normalized",
    "canceled_discipline", "skipped_discipline"
]
treated_df = treated_df[cols]

# Dicionário de renomeamento
rename_map = {
    "birthdate_normalized": "birthdate",
    "sex_normalized": "sex",
    "course_normalized": "course",
    "period_normalized": "period",
    "week_day_normalized": "week_day",
    "discipline_normalized": "discipline",
    "status_normalized": "status",
    "g1_normalized": "g1",
    "g2_normalized": "g2",
    "final_grade_normalized": "final_grade"
}

treated_df = treated_df.rename(columns=rename_map)

treated_df.head()


,id,student_id,birthdate,sex,city,course,period,week_day,discipline,status,g1,g2,final_grade,canceled_discipline,skipped_discipline
2138,1,99951,1992-05-21,2,Faxinal do Soturno,1,1,1,Matemática Aplicada,1,0.75,0.70,0.72,0,0
5665,3,99951,1992-05-21,2,Faxinal do Soturno,1,2,2,Contabilidade Intermediária,1,0.72,0.89,0.80,0,0
9160,2,99951,1992-05-21,2,Faxinal do Soturno,1,3,3,Contabilidade Avançada,0,0.00,0.00,0.00,1,0
28710,4,99955,1989-01-06,1,Faxinal do Soturno,1,4,2,Contabilidade Introdutória,1,0.84,0.88,0.86,0,0
28312,8,99955,1989-01-06,1,Faxinal do Soturno,1,4,1,Legislação e Ética Profissional,1,0.98,0.99,0.98,0,0


In [6]:
# Salva dataset com tratamentos apenas
treated_df.to_csv("datasets/original_treated.csv", index=False, encoding="utf-8")